<a href="https://colab.research.google.com/github/HaJunYoo/Pyspark-tutorial/blob/main/mllib_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 
!pip install -q findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "logistic-regression")
conf.set("spark.master", "local[*]")

# Singleton pattern
spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=bf7c25c8070f1d8ddb7c2df0b24321605dc60cecfecaf3d28869889e45165548
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


- 파이프라인은 데이터의 여러 가지 처리 단계를 거칠 때 유용하다
- 예) nlp의 토큰나이제이션 -> 모델링

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer # termfrequency, tokenizer

In [4]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

In [6]:
# tokenizer 설정
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# tokenizer에서 토큰화된 vocab의 term frequency 생성
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

In [7]:
lr = LogisticRegression(maxIter=30, regParam=0.001)

In [8]:
# 파이프라인 단계를 설정
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [9]:
model = pipeline.fit(training)

In [10]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [11]:
prediction = model.transform(test)

In [12]:
prediction.select(["id", "text", "probability", "prediction"]).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.63102699631690...|       0.0|
|  5|             l m n|[0.98489377609773...|       0.0|
|  6|spark hadoop spark|[0.13563147748817...|       1.0|
|  7|     apache hadoop|[0.99563405823116...|       0.0|
+---+------------------+--------------------+----------+

